# High mem issue
This nb is to reproduce an issue where generating negative edges consumes a large
amount of memory and requires a long time

Graph used in the NB is available here:
http://doi.org/10.5281/zenodo.4011267

In [7]:
# get graph and embeddings
graph_data_dir = "graph"
import urllib
import os
os.makedirs(graph_data_dir, exist_ok=True)
if not os.path.exists(graph_data_dir + "/kg-covid-19-skipgram-aug-2020.tar.gz"):
    with urllib.request.urlopen("https://zenodo.org/record/4011267/files/kg-covid-19-skipgram-aug-2020.tar.gz") as response, \
        open(graph_data_dir + "/kg-covid-19-skipgram-aug-2020.tar.gz", 'wb') as out_file:
            data = response.read()  # a `bytes` object
            out_file.write(data)
os.system("tar -xvzf " + graph_data_dir + "/kg-covid-19-skipgram-aug-2020.tar.gz -C " + graph_data_dir)

0

## Loading the graph

In [ ]:
from ensmallen_graph import EnsmallenGraph

graph = EnsmallenGraph.from_csv(
    edge_path=os.path.join(graph_data_dir, "merged-kg_edges.tsv"),
    sources_column="subject",
    destinations_column="object",
    directed=False,
    edge_types_column="edge_label",
    default_edge_type="biolink:association",
    node_path=os.path.join(graph_data_dir, "merged-kg_nodes.tsv"),
    nodes_column="id",
    node_types_column="category",
    default_node_type="biolink:NamedThing",
    ignore_duplicated_edges=True,
    ignore_duplicated_nodes=True,
    force_conversion_to_undirected=True
)

In [ ]:
graph.report()

In [ ]:
# memory of sample_negatives() seems to peak at or around 200G in my hands
seed = 42
neg_graph = graph.sample_negatives(
   seed=seed,
   negatives_number=graph.get_edges_number(),
   allow_selfloops=False
)
neg_training, neg_validation = neg_graph.random_holdout(seed=seed, train_percentage=train_percentage)